# Import

In [1]:
#!pip install tensorflow opencv-python mediapipe==0.10.11 scikit-learn matplotlib
#python 3.8.0

In [2]:
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Using mediapipe (mp)

In [30]:
mp_holistic = mp.solutions.holistic #holistic model means that it will include hands, face, body detection
mp_drawing = mp.solutions.drawing_utils #for arranging landmarks

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #opencv take frames as BGR but mediapipe takes RGB therefore have to convert
    image.flags.writeable = False #prevent from changing image while mp.process (only readable)
    results = model.process(image) #process (detects hands face body)
    image.flags.writeable = True #make writeable again after mp.process
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #convert the color again because opencv will display the image

    return image, results

def draw_landmarks(image, results):
    if results.pose_landmarks: #if body in the image, make draw landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    if results.face_landmarks:#if face in the image, make draw landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)

    if results.left_hand_landmarks:#if left hand in the image, make draw landmarks
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    if results.right_hand_landmarks:#if right in the image, make draw landmarks
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

def show_preprocessed_video(model):
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read() #take frame and ret returns True if camera read is sucsessfully done else False

        image, results = mediapipe_detection(frame, model) #detect hand face body

        draw_landmarks(image, results) #draw landmarks

        cv2.imshow("MediaPipe processed video", image) #display frame 


        if cv2.waitKey(1) & 0xFF == ord("q"):#q = break and wait 1ms
            break

    cap.release() #relase the cam
    cv2.destroyAllWindows() #close the display screen 


In [ ]:
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) #create the model that recognize face, hands, body

show_preprocessed_video(holistic)
